In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image
Image.open('../input/hot-dog-not-hot-dog/test/hot_dog/133012.jpg').size

In [ ]:
Image.open('../input/hot-dog-not-hot-dog/test/hot_dog/133015.jpg').size

In [ ]:
load_img('../input/hot-dog-not-hot-dog/test/hot_dog/133012.jpg')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_tr = ImageDataGenerator(rescale = 1.0/255).flow_from_directory(
            '../input/hot-dog-not-hot-dog/train',
            target_size = (224, 224), seed = 123)

data_te = ImageDataGenerator(rescale = 1.0/255).flow_from_directory(
            '../input/hot-dog-not-hot-dog/test',
            target_size = (224, 224), seed = 123)

In [ ]:
# combine both datasets 
data_tr.class_indices

In [ ]:
# split images and labels 
images_tr, labels_tr = next(data_tr)
images_te, labels_te = next(data_te)

In [ ]:
images = np.concatenate((images_tr, images_te))
labels = np.concatenate((labels_tr[:,0], labels_te[:,0]))

In [ ]:
for i in range(5):
    plt.imshow(images[i])
    plt.title(labels[i])
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_model, X_test, y_model, y_test = train_test_split(images, labels, test_size=0.20, random_state=123)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
cnn = Sequential()
cnn.add(Conv2D(64, (3, 3),
                      activation='relu',
                      input_shape=(224, 224,  3),
                      padding='SAME'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(32, (3, 3), activation='relu', padding='SAME'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Flatten())
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

cnn.compile(loss='binary_crossentropy',
            optimizer="adam",
            metrics=['accuracy'])
cnn.summary()

China just released a model with 1.75 trillion trainable parameters 

In [ ]:
cnn1 = cnn.fit(X_model,
               y_model,
               epochs=100,
               batch_size=50,
               validation_data=(X_test, y_test))

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=15, min_delta=0.001, verbose=3),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
cnn1 = cnn.fit(X_model,
               y_model,
               epochs=100,
               batch_size=50,
               validation_data=(X_test, y_test), 
               callbacks = callbacks)

In [ ]:
from keras.applications import InceptionV3
from keras.layers import GlobalAveragePooling2D

In [ ]:
imagenet = InceptionV3(weights='imagenet', include_top=False)
imagenet_new = imagenet.output
new_model = Sequential()
new_model.add(imagenet)
new_model.add(GlobalAveragePooling2D())
new_model.add(Dense(2048, activation='relu'))
new_model.add(Dense(1024, activation='relu'))  # dense layer 2
new_model.add(Dense(512, activation='relu'))  # dense layer 3
# final layer with softmax activation
new_model.add(Dense(1, activation='sigmoid'))

In [ ]:
for i, layer in enumerate(imagenet.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
for i,layer in enumerate(new_model.layers):
    print(i,layer.name, layer.trainable)

In [ ]:
for layer in new_model.layers[:1]:
    layer.trainable=False

In [ ]:
for i,layer in enumerate(new_model.layers):
    print(i,layer.name, layer.trainable)

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(optimizer='Adam',
                  loss='binary_crossentropy', metrics=['accuracy'])

# step_size_train=train_generator.n//train_generator.batch_size
transfer = new_model.fit(X_model,
              y_model,
              epochs=50,
              batch_size=128,
              validation_data=(X_test, y_test),
              callbacks = callbacks)

In [ ]:
# looks like we are overfitting 
imagenet = InceptionV3(weights='imagenet', include_top=False)
imagenet_new = imagenet.output
new_model = Sequential()
new_model.add(imagenet)
new_model.add(GlobalAveragePooling2D())
new_model.add(Dense(1024, activation='relu'))  # dense layer 2
new_model.add(Dense(512, activation='relu'))  # dense layer 3
# final layer with softmax activation
new_model.add(Dense(1, activation='sigmoid'))

In [ ]:
new_model.summary()

In [ ]:
for layer in new_model.layers[:1]:
    layer.trainable=False
new_model.summary()

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=3, min_delta=0.001, verbose=3),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
new_model.compile(optimizer='Adam',
                  loss='binary_crossentropy', metrics=['accuracy'])

# step_size_train=train_generator.n//train_generator.batch_size
transfer = new_model.fit(X_model,
              y_model,
              epochs=50,
              batch_size=128,
              validation_data=(X_test, y_test),
              callbacks = callbacks)

In [ ]:
# Gradcam 